In [1]:
import pandas as pd
import numpy as np
import pickle as pk
import nibabel as nib
import random
import sys
import os.path
import glob

In [2]:
control_p = ['P054','P057','P064','P065','P067','P068','P072','P073','P075','P076','P080','P081']
ASD_p = ['P050','P055','P056','P058','P059','P060','P066','P069','P070','P071','P078','P079']

#Participants missing in Brea's P060, P066
all_participants = sorted(ASD_p + control_p)
print("Number of participants: ",len(all_participants))

csv_dir = '/home/varshini/projects/def-afyshe-ab/varshini/glucks/data/stim_order_160/'
csvs = sorted(glob.glob(csv_dir+'*.csv')) 

csv_dict = {}
for csv in csvs:
    
    participant_name = csv[-8:-4]
    
    if participant_name in all_participants:
        csv_dict[participant_name] = csv

Number of participants:  24


In [3]:
def process_csvs(participant):
    
    beta_dir = '/home/varshini/scratch/data/data_glucksberg/betas_alphabetical_ordered/'
    beta_path = beta_dir + participant + '/1st_Level_ROI_alph/'

    beta_paths = [beta_path + "beta_{:04d}.nii".format(i) for i in range(1, 161)]

    beta_array = []
    for beta_file in beta_paths:
        beta_img = nib.load(beta_file)

        beta_img_np = np.array(beta_img.get_fdata())
        beta_img_np[np.isnan(beta_img_np)] = 0
        beta_img_np = beta_img_np.flatten().tolist()
        beta_array.append(beta_img_np)

    out_dir = '/home/varshini/scratch/data/data_glucksberg/processed_data/csvs/'

    df = pd.read_csv(csv_dict[participant])
    df = df[['stim','SentType','StimOnset','run','category','exemplar']]
    df = df.sort_values(by='stim',ignore_index = True) 
    df['betas'] = beta_array

    all_stim_path = out_dir + 'all_stim_csvs/'
    os.makedirs(all_stim_path, exist_ok=True)  
    df.to_csv(all_stim_path + participant +'.csv')  

    df_lt = df[df['SentType'] == 'LT']
    df_lt = df_lt.reset_index(drop=True)
    lt_path = out_dir + 'lt_csvs/'
    os.makedirs(lt_path, exist_ok=True)
    df_lt.to_csv(lt_path + participant +'.csv') 

    df_lf = df[df['SentType'] == 'LF']
    df_lf = df_lf.reset_index(drop=True)
    lf_path = out_dir + 'lf_csvs/'
    os.makedirs(lf_path, exist_ok=True)
    df_lf.to_csv(lf_path + participant +'.csv') 

    df_m = df[df['SentType'] == 'M']
    df_m = df_m.reset_index(drop=True)
    m_path = out_dir + 'm_csvs/'
    os.makedirs(m_path, exist_ok=True)
    df_m.to_csv(m_path + participant +'.csv') 

    df_sm = df[df['SentType'] == 'SM']
    df_sm = df_sm.reset_index(drop=True)
    sm_path = out_dir + 'sm_csvs/'
    os.makedirs(sm_path, exist_ok=True)
    df_sm.to_csv(sm_path + participant +'.csv') 

In [ ]:
for participant in all_participants:
        process_csvs(participant)

In [4]:
def read_csvs(participant):
    
    csv_dir = '/home/varshini/scratch/data/data_glucksberg/processed_data/csvs/'

    all_stim_path = csv_dir + 'all_stim_csvs/' 
    df = pd.read_csv(all_stim_path + participant +'.csv')  

    lt_path = csv_dir + 'lt_csvs/'
    df_lt = pd.read_csv(lt_path + participant +'.csv') 
    
    lf_path = csv_dir + 'lf_csvs/'
    df_lf = pd.read_csv(lf_path + participant +'.csv') 
    
    m_path = csv_dir + 'm_csvs/'
    df_m = pd.read_csv(m_path + participant +'.csv') 
    
    sm_path = csv_dir + 'sm_csvs/'
    df_sm = pd.read_csv(sm_path + participant +'.csv')
    
    return df, df_lt, df_lf, df_sm, df_m

In [5]:
def save_betas(participant, beta_out_dir):
    
    df, df_lt, df_lf, df_sm, df_m = read_csvs(participant)
    
    all_betas = df['betas'].to_list()
    all_betas = [eval(all_betas[i]) for i in range(len(all_betas))]
    all_betas = np.array(all_betas)
    with open(beta_out_dir + 'all_stim_betas/' + participant + '.pkl','wb') as f:
        pk.dump(all_betas, f)
    
    lt_betas = df_lt['betas'].to_list()
    lt_betas = [eval(lt_betas[i]) for i in range(len(lt_betas))]
    lt_betas = np.array(lt_betas)
    with open(beta_out_dir + 'lt_betas/' + participant + '.pkl','wb') as f:
        pk.dump(lt_betas, f)
    
    lf_betas = df_lf['betas'].to_list()
    lf_betas = [eval(lf_betas[i]) for i in range(len(lf_betas))]
    lf_betas = np.array(lf_betas)
    with open(beta_out_dir + 'lf_betas/' + participant + '.pkl','wb') as f:
        pk.dump(lf_betas, f)
    
    m_betas = df_m['betas'].to_list()
    m_betas = [eval(m_betas[i]) for i in range(len(m_betas))]
    m_betas = np.array(m_betas)
    with open(beta_out_dir + 'm_betas/' + participant + '.pkl','wb') as f:
        pk.dump(m_betas, f)
    
    sm_betas = df_sm['betas'].to_list()
    sm_betas = [eval(sm_betas[i]) for i in range(len(sm_betas))]
    sm_betas = np.array(sm_betas)
    with open(beta_out_dir + 'sm_betas/' + participant + '.pkl','wb') as f:
        pk.dump(sm_betas, f)

In [ ]:
beta_out_dir = '/home/varshini/scratch/data/data_glucksberg/processed_data/betas/alphabetical/'
for participant in all_participants:
    print(participant)
    save_betas(participant, beta_out_dir)

P050
P054
P055
P056
P057
P058
P059
P060
P064
P065
P066
P067
P068
P069
P070
P071
P072
P073
P075
P076
P078
P079
P080
